**Query** <br>
Individuare per il nord, il centro e il sud le tre regioni che hanno effettuato piu tamponi dall’inizio della pandemia. Per determinare le regioni, calcolare la media aritmetica dei tamponi effettuati ogni giorno dall’inizio della pandemia. <br> <br>
Suggerimento: per determinare le regioni appartenenti al nord, centro e sud, e possibile consultare liste staticamente definite. In alternativa, si possono utilizzare le coordinate di latitudine e longitudine fornite nel dataset. 

**Dataset** <br>
Il dataset è organizzato secondo il seguente schema: <br>
data | stato| codice_regione | denominazione_regione | lat | long | ricoverati_con_sintomi | terapia_intensiva | totale_ospedalizzati | isolamento_domiciliare | totale_positivi | variazione_totale_positivi | nuovi_positivi | dimessi_guariti | deceduti | totale_casi | tamponi | casi_testati | note_it | note_en

data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note

In [1]:
%%capture
!pip install pyspark

In [2]:
from pyspark import SparkContext

In [3]:
# SUDDIVISIONE ISTAT
mydict = {"Liguria":"nord", "Lombardia":"nord", "Piemonte":"nord", "Valle d'Aosta":"nord", "Emilia-Romagna":"nord", "Friuli Venezia Giulia":"nord", "P.A. Bolzano":"nord", "P.A. Trento":"nord", "Veneto":"nord", "Lazio":"centro", "Marche":"centro", "Toscana":"centro", "Umbria":"centro", "Abruzzo":"sud", "Basilicata":"sud", "Calabria":"sud", "Campania":"sud", "Molise":"sud", "Puglia":"sud", "Sardegna":"sud", "Sicilia":"sud"}

In [4]:
# DOWNLOAD FILE CSV
!wget https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv

--2021-01-18 10:17:33--  https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 974734 (952K) [text/plain]
Saving to: ‘dpc-covid19-ita-regioni.csv’

dpc-covid19-ita-reg 100%[===================>] 951.89K  --.-KB/s    in 0.07s   

2021-01-18 10:17:33 (13.4 MB/s) - ‘dpc-covid19-ita-regioni.csv’ saved [974734/974734]



In [5]:
sc = SparkContext("local[2]", "covid3")

In [6]:
# IMPORTO DATASET RIMUOVENDO L'HEADER
rawData = sc.textFile("dpc-covid19-ita-regioni.csv")
header = rawData.first()
rawData = rawData.filter(lambda line: line != header)

In [7]:
rawData.take(5)

['2020-02-24T18:00:00,ITA,13,Abruzzo,42.35122196,13.39843823,0,0,0,0,0,0,0,0,0,,,0,5,,,,,',
 '2020-02-24T18:00:00,ITA,17,Basilicata,40.63947052,15.80514834,0,0,0,0,0,0,0,0,0,,,0,0,,,,,',
 '2020-02-24T18:00:00,ITA,18,Calabria,38.90597598,16.59440194,0,0,0,0,0,0,0,0,0,,,0,1,,,,,',
 '2020-02-24T18:00:00,ITA,15,Campania,40.83956555,14.25084984,0,0,0,0,0,0,0,0,0,,,0,10,,,,,',
 '2020-02-24T18:00:00,ITA,08,Emilia-Romagna,44.49436681,11.341720800000001,10,2,12,6,18,0,18,0,0,,,18,148,,,,,']

In [8]:
data = rawData.map(lambda line:line.split(',')).map(lambda x : (str(x[3]), int(x[18])))

In [9]:
data.take(10)

[('Abruzzo', 5),
 ('Basilicata', 0),
 ('Calabria', 1),
 ('Campania', 10),
 ('Emilia-Romagna', 148),
 ('Friuli Venezia Giulia', 58),
 ('Lazio', 124),
 ('Liguria', 1),
 ('Lombardia', 1463),
 ('Marche', 16)]

In [10]:
query = data.groupByKey().mapValues(sorted).map(lambda x: (mydict[x[0]],(round(max(x[1])/len(x[1]),1), x[0]))).groupByKey().mapValues(sorted).map(lambda x : (x[0], x[1][:-4:-1]))

In [11]:
query.collect()

[('nord',
  [(15903.3, 'Lombardia'), (11075.2, 'Veneto'), (8542.1, 'Emilia-Romagna')]),
 ('centro', [(9004.6, 'Lazio'), (6199.1, 'Toscana'), (1879.7, 'Marche')]),
 ('sud', [(6801.8, 'Campania'), (4369.6, 'Sicilia'), (3591.9, 'Puglia')])]

In [12]:
query.coalesce(1).saveAsTextFile("output_covid3")
sc.stop()

In [13]:
!ls

dpc-covid19-ita-regioni.csv  output_covid3  sample_data


In [14]:
!cat output_covid3/part-00000

('nord', [(15903.3, 'Lombardia'), (11075.2, 'Veneto'), (8542.1, 'Emilia-Romagna')])
('centro', [(9004.6, 'Lazio'), (6199.1, 'Toscana'), (1879.7, 'Marche')])
('sud', [(6801.8, 'Campania'), (4369.6, 'Sicilia'), (3591.9, 'Puglia')])


In [15]:
!rm -r output_covid3